###### Imports and Settings

In [14]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [15]:
#read in API key
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [16]:
#variable containing Census API key
api_key = keys_dict_2['CENSUS']

## Read In Data Guide

In [17]:
dataguide = pd.read_csv('../../Data Guides/DATA GUIDE ACS 2020_2021 5YR BG.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [18]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]
dg3 = dataguide[dataguide['ID'].between(93, 138)]
dg4 = dataguide[dataguide['ID'].between(139, 184)]
dg5 = dataguide[dataguide['ID'].between(185, 230)]
dg6 = dataguide[dataguide['ID'].between(231, 276)]
dg7 = dataguide[dataguide['ID'].between(277, 322)]
dg8 = dataguide[dataguide['ID'].between(323, 368)]
dg9 = dataguide[dataguide['ID'].between(369, 414)]
dg10 = dataguide[dataguide['ID'].between(415, 460)]
dg11 = dataguide[dataguide['ID'].between(461, 506)]
dg12 = dataguide[dataguide['ID'].between(507, 552)]
dg13 = dataguide[dataguide['ID'].between(553, 598)]
dg14 = dataguide[dataguide['ID'].between(599, 644)]
dg15 = dataguide[dataguide['ID'].between(645, 690)]
dg16 = dataguide[dataguide['ID'].between(691, 736)]
dg17 = dataguide[dataguide['ID'].between(737, 782)]
dg18 = dataguide[dataguide['ID'].between(783, 828)]
dg19 = dataguide[dataguide['ID'].between(829, 874)]
dg20 = dataguide[dataguide['ID'].between(875, 920)]
dg21 = dataguide[dataguide['ID'].between(921, 966)]
dg22 = dataguide[dataguide['ID'].between(967, 1012)]
dg23 = dataguide[dataguide['ID'].between(1013, 1058)]
dg24 = dataguide[dataguide['ID'].between(1059, 1104)]
dg25 = dataguide[dataguide['ID'].between(1105, 1150)]
dg26 = dataguide[dataguide['ID'].between(1151, 1196)]
dg27 = dataguide[dataguide['ID'].between(1197, 1242)]
dg28 = dataguide[dataguide['ID'].between(1243, 1287)]
dg29 = dataguide[dataguide['ID'].between(1288, 1332)]
dg30 = dataguide[dataguide['ID'].between(1333, 1377)]
dg31 = dataguide[dataguide['ID'].between(1378, 1422)]
dg32 = dataguide[dataguide['ID'].between(1423, 1467)]
dg33 = dataguide[dataguide['ID'].between(1468, 1512)]
dg34 = dataguide[dataguide['ID'].between(1513, 1557)]
dg35 = dataguide[dataguide['ID'].between(1558, 1602)]
dg36 = dataguide[dataguide['ID'].between(1603, 1647)]
dg37 = dataguide[dataguide['ID'].between(1648, 1692)]
dg38 = dataguide[dataguide['ID'].between(1693, 1737)]
dg39 = dataguide[dataguide['ID'].between(1738, 1782)]
dg40 = dataguide[dataguide['ID'].between(1783, 1827)]
dg41 = dataguide[dataguide['ID'].between(1828, 1872)]
dg42 = dataguide[dataguide['ID'].between(1873, 1917)]
dg43 = dataguide[dataguide['ID'].between(1918, 1962)]
dg44 = dataguide[dataguide['ID'].between(1963, 2007)]

In [19]:
dfs = [dg1, dg2, dg3, dg4, dg5, dg6, dg7, dg8, dg9, dg10, dg11, dg12, dg13, dg14, dg15, dg16, dg17, dg18, dg19, dg20, dg21, dg22, dg23, dg24, dg25, 
       dg26, dg27, dg28, dg29, dg30, dg31, dg32, dg33, dg34, dg35, dg36, dg37, dg38, dg39, dg40, dg41, dg42, dg43, dg44]
dfs = [dg3, dg4]

In [20]:
#url string and list parameters for column head and tail
url_str= 'https://api.census.gov/data/2021/acs/acs5?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'CountyFIPS'
tail_cols3 = 'CensusTract'
tail_cols4 = 'BlockGroup'

In [21]:
results = []
for df in dfs:
    dataguide = df
    for col_name, col_data in df.items():
        var_list = list(dataguide['Variable']) #make variables list
        var_list = deque(var_list)
        var_list.appendleft(head2)
        var_list.appendleft(head1)
        col_list = list(dataguide['Column Name']) #make columns list
        col_list.append(tail_cols1)
        col_list.append(tail_cols2)
        col_list.append(tail_cols3)
        col_list.append(tail_cols4)
        col_list = deque(col_list)
        col_list.appendleft(head2)
        col_list.appendleft(head1)
        predicates= {} #block groups Tennessee
        get_vars= var_list
        predicates["get"]= ",". join(get_vars)
        predicates["for"]= "block group:*"
        predicates["in"]= "state:47, county:*, tract:*"
        data = requests.get(url_str, params = predicates)
        col_names = col_list
        df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
        results.append(df)
new_df = pd.concat(results, axis = 1)
new_df = new_df.transpose().reset_index(drop = False).drop_duplicates()
new_df.columns = new_df.iloc[0]
new_df = new_df.set_index('NAME').transpose().drop(columns = ['StateFIPS','CountyFIPS', 'CensusTract', 'BlockGroup']).reset_index(drop = True)
new_df = new_df.rename_axis(None, axis = 1)
print('Okay Finished')

Okay Finished


In [35]:
new_df.head(2)

,NAME,GEO_ID,housingcost_total_selectedownercosts%hhincome_series,housingcost_total%ownercostwmortgage_series,housingcost_%ownercost30to34.9_wmortgage,housingcost_%ownercost35to39.9_wmortgage,housingcost_%ownercost40to49.9_wmortgage,housingcost_%ownercost50+_wmortgage,housingcost_total%ownercostwomortgage_series,housingcost_%ownercost30to34.9_womortgage,housingcost_%ownercost35to39.9_womortgage,housingcost_%ownercost40to49.9_womortgage,housingcost_%ownercost50+_womortgage,housingcost_total_rent%hhincome_series,housingcost_%rentercost30to34.9,housingcost_%rentercost35to39.9,housingcost_%rentercost40to49.9,housingcost_%rentercost50+,housing_medianvalue,housingcost_medvalue_ownerocc,housingcost_medvalue_ownerocc_wmortgage,housingcost_medvalue_ownerocc_womortgage,housingcost_mediangrossrent_renteroccupied,structures_total_yearbuilt_series,structures_built2014orlater,structures_built2010to2013,structures_built2000to2009,structures_built1990to1999,structures_built1980to1989,structures_built1970to1979,structures_built1960to1969,structures_built1950to1959,structures_built1940to1949,structures_built1939orearlier,structures_medianyearbuilt,commute_total_meansoftransportationtowork_series,commute_cartruckvan,commute_cartruckvan_drovealone,commute_cartruckvan_carpooled,commute_cartruckvan_carpooled_2ppl,commute_cartruckvan_carpooled_3ppl,commute_cartruckvan_carpooled_4ormoreppl,commute_publictransportation,commute_publictransportation_bus,commute_publictransportation_subwayorelevatedrail,commute_publictransportation_longdistancetrainorcommuterrail,commute_publictransportation_lightrailstreetcarortrolley,commute_publictransportation_ferryboat,commute_bicycle,commute_walk,commute_taxicabmotorcycleother,commute_workedfromhome,vehicles_total_series2,vehicles_none,vehicles_one,vehicles_two,vehicles_three,vehicles_four,vehicles_fiveormore,foreignborn_total,fb_europe,fb_eur_northern,fb_eur_n_denmark,fb_eur_n_ireland,fb_eur_n_norway,fb_eur_n_sweden,fb_eur_n_uk,fb_eur_n_excluding england and scotland,fb_eur_n_england,fb_eur_n_scotland,fb_eur_n_other,fb_eur_western,fb_eur_w_austria,fb_eur_w_belgium,fb_eur_w_france,fb_eur_w_germany,fb_eur_w_netherlands,fb_eur_w_switzerland,fb_eur_w_other,fb_eur_southern,fb_eur_s_greece,fb_eur_s_italy,fb_eur_s_portugal,fb_eur_s_azoresislands,fb_eur_s_spain,fb_eur_s_other,fb_eur_eastern,fb_eur_e_albania,fb_eur_e_belarus,fb_eur_e_bosniaandherzegovina,fb_eur_e_bulgaria,fb_eur_e_croatia,fb_eur_e_czechoslovakia,fb_eur_e_hungary
0,"Block Group 1, Census Tract 201, Anderson Coun...",1500000US470010201001,153,108,0,0,0,0,45,0,6,0,0,797,38,74,22,132,314900,None,None,None,801,1112,45,23,14,48,32,86,57,219,572,16,1949,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"Block Group 2, Census Tract 201, Anderson Coun...",1500000US470010201002,271,133,0,12,17,12,138,0,0,0,0,347,37,41,36,118,202500,None,None,None,892,727,0,14,150,78,155,143,76,35,76,0,1982,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [36]:
poverty = new_df[['NAME', 'GEO_ID', #'poverty_totalinhh', 'poverty_totalinhh_belowlevel', 
                  #'housing_medianvalue'
#                   'commute_total_meansoftransportationtowork_series', 
#                   'commute_workedfromhome', 
                  'housingcost_total_selectedownercosts%hhincome_series', 'housingcost_total%ownercostwmortgage_series', 'housingcost_%ownercost30to34.9_wmortgage', 
                  'housingcost_%ownercost35to39.9_wmortgage', 'housingcost_%ownercost40to49.9_wmortgage', 'housingcost_%ownercost50+_wmortgage', 
                  'housingcost_total%ownercostwomortgage_series', 'housingcost_%ownercost30to34.9_womortgage', 'housingcost_%ownercost35to39.9_womortgage', 
                  'housingcost_%ownercost40to49.9_womortgage', 'housingcost_%ownercost50+_womortgage', 'housingcost_total_rent%hhincome_series', 
                  'housingcost_%rentercost30to34.9', 'housingcost_%rentercost35to39.9', 'housingcost_%rentercost40to49.9', 'housingcost_%rentercost50+'
                 ]]
poverty = poverty.set_index(['NAME', 'GEO_ID'])
cols = poverty.columns
poverty[cols] = poverty[cols].astype(float)
#poverty['Percent in Households Below Poverty Level'] = (poverty['poverty_totalinhh_belowlevel']/poverty['poverty_totalinhh'])*100

In [37]:
poverty.reset_index(drop = False, inplace = True)

In [38]:
poverty['GEO_ID'] = poverty['GEO_ID'].str[9:]

In [43]:
thelist = [poverty['housingcost_%ownercost30to34.9_wmortgage'], poverty['housingcost_%ownercost35to39.9_wmortgage'], 
           poverty['housingcost_%ownercost40to49.9_wmortgage'], poverty['housingcost_%ownercost50+_wmortgage'], 
           poverty['housingcost_%ownercost30to34.9_womortgage'], poverty['housingcost_%ownercost35to39.9_womortgage'], 
           poverty['housingcost_%ownercost40to49.9_womortgage'], poverty['housingcost_%ownercost50+_womortgage']]
ownerscostburdened = sum(thelist)
poverty['Percent Owners Cost Burdened'] = (sum(thelist)/poverty['housingcost_total_selectedownercosts%hhincome_series'])*100

thelist = [poverty['housingcost_%rentercost30to34.9'], poverty['housingcost_%rentercost35to39.9'], 
           poverty['housingcost_%rentercost40to49.9'], poverty['housingcost_%rentercost50+']]
renterscostburdened = sum(thelist)
poverty['Percent Owners Cost Burdened'] = (sum(thelist)/poverty['housingcost_total_rent%hhincome_series'])*100

totalseries = poverty['housingcost_total_selectedownercosts%hhincome_series']+poverty['housingcost_total_rent%hhincome_series']

poverty['Percent Cost Burdened'] = (((ownerscostburdened+renterscostburdened)/totalseries)*100)

In [44]:
poverty = poverty[['NAME', 'GEO_ID', 'Percent Owners Cost Burdened', 'Percent Owners Cost Burdened', 'Percent Cost Burdened']]

In [45]:
poverty.head()

,NAME,GEO_ID,Percent Owners Cost Burdened,Percent Owners Cost Burdened,Percent Cost Burdened
0,"Block Group 1, Census Tract 201, Anderson Coun...",470010201001,33.375157,33.375157,28.631579
1,"Block Group 2, Census Tract 201, Anderson Coun...",470010201002,66.858790,66.858790,44.174757
2,"Block Group 1, Census Tract 202.01, Anderson C...",470010202011,23.640167,23.640167,17.638984
3,"Block Group 2, Census Tract 202.01, Anderson C...",470010202012,20.338983,20.338983,13.362069
4,"Block Group 1, Census Tract 202.02, Anderson C...",470010202021,NaN,NaN,6.551060


In [46]:
poverty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4562 entries, 0 to 4561
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   NAME                          4562 non-null   object 
 1   GEO_ID                        4562 non-null   object 
 2   Percent Owners Cost Burdened  4407 non-null   float64
 3   Percent Owners Cost Burdened  4407 non-null   float64
 4   Percent Cost Burdened         4527 non-null   float64
dtypes: float64(3), object(2)
memory usage: 178.3+ KB


In [47]:
poverty.to_csv('costburdenbg.csv')

In [12]:
data = new_df

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4562 entries, 0 to 4561
Columns: 3039 entries, NAME to earn_fulltime_ median_test2
dtypes: object(3039)
memory usage: 105.8+ MB


In [15]:
data.set_index(['NAME', 'GEO_ID'], inplace = True)

In [21]:
data.tail()

,,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,units_total_series,units_one_detached,units_one_attached,units_two,units_threetofour,units_fivetonine,units_tentonineteen,units_twentytofortynine,units_fiftyormore,units_mobilehome,units_boatrvvanetc,hhtype_total_series,hhtype_familyhh,hhtype_familyhh_marriedcouplefam,hhtype_familyhh_otherfam,hhtype_familyhh_malenospouse,hhtype_familyhh_femalenospouse,hhtype_nonfamhh,hhtype_nonfamhh_householderalone,hhtype_nonfamhh_householdernotalone,hhincome_median,hhincome_total_series,hhincome_lessthan10000,hhincome_10to14999,hhincome_15to19999,hhincome_20to24999,hhincome_25to29999,hhincome_30to34999,hhincome_35to39999,hhincome_40to44999,hhincome_45to49999,hhincome_50to59999,hhincome_60to74999,hhincome_75to99999,hhincome_100to124999,hhincome125to149999,hhincome150to199999,hhincome200ormore,housingcost_total_selectedownercosts%hhincome_series,housingcost_total%ownercostwmortgage_series,housingcost_%ownercost30to34.9_wmortgage,housingcost_%ownercost35to39.9_wmortgage,housingcost_%ownercost40to49.9_wmortgage,housingcost_%ownercost50+_wmortgage,housingcost_total%ownercostwomortgage_series,housingcost_%ownercost30to34.9_womortgage,housingcost_%ownercost35to39.9_womortgage,housingcost_%ownercost40to49.9_womortgage,housingcost_%ownercost50+_womortgage,housingcost_total_rent%hhincome_series,housingcost_%rentercost30to34.9,housingcost_%rentercost35to39.9,housingcost_%rentercost40to49.9,housingcost_%rentercost50+,housing_medianvalue,housingcost_mediangrossrent_renteroccupied,structures_total_yearbuilt_series,structures_built2014orlater,structures_built2010to2013,structures_built2000to2009,structures_built1990to1999,structures_built1980to1989,structures_built1970to1979,structures_built1960to1969,structures_built1950to1959,structures_built1940to1949,structures_built1939orearlier,structures_medianyearbuilt,attainment_total_over25_series,attainment_noschooling,attainment_nurseryschool,attainment_kindergarten,attainment_1stgrade,attainment_2ndgrade,attainment_3rdgrade,attainment_4thgrade,attainment_5thgrade,attainment_6thgrade,attainment_7thgrade,attainment_8thgrade,attainment_9thgrade,attainment_10thgrade,attainment_11thgrade,attainment_12thgradenodiploma,attainment_regularhighschooldiploma,attainment_gedoralternativecredential,attainment_somecollegelessthan1year,attainment_somecollege1ormoreyearsnodegree,attainment_associatesdegree,attainment_bachelorsdegree,attainment_mastersdegree,attainment_professionalschooldegree,attainment_doctoratedegree,vehicles_tenurebyvehicles_total_series,vehicles_ownerocc_total,vehicles_ownerocc_novehicle,vehicles_ownerocc_1vehicle,vehicles_ownerocc_2vehicles,vehicles_ownerocc_3vehicles,vehicles_ownerocc_4vehicles,vehicles_ownerocc_5+vehicles,vehicles_renterocc_total,vehicles_renterocc_novehicle,vehicles_renterocc_1vehicle,vehicles_renterocc_2vehicles,vehicles_renterocc_3vehicles,vehicles_renterocc_4vehicles,vehicles_renterocc_5+vehicles,industry_sexbyindustrycivilianpop_total_series,industry_sexbyindustrycivilianpop_totalmale,industry_m_agriculture_forestry_fishing_hunting_mining,industry_m_agmining_agriculture_forestry_fishing_hunting,industry_m_agmining_mining_quarrying_oilandgas,industry_m_construction,industry_m_manufacturing,industry_m_wholesaletrade,industry_m_retailtrade,industry_m_transportation_warehousing_util

In [20]:
data = data.dropna(axis = 1)

In [22]:
cols = data.columns
data[cols] = data[cols].astype(float)

In [24]:
data = data.reset_index(drop = False)

In [27]:
data.to_feather('../../Raw Data/ACS20215YRBG')